In [1]:
%matplotlib widget

import logging
import time

import cftime
import httpx
import matplotlib.pyplot as plt
import netCDF4
import pandas as pd
import shapely.io
import sqlmodel
from sqlalchemy import (
    bindparam, 
    func
)

from arpav_ppcv import (
    database as db,
    operations,
)
from arpav_ppcv.config import get_settings
from arpav_ppcv.schemas.base import (
    CoverageDataSmoothingStrategy,
    ObservationDataSmoothingStrategy,
    ObservationAggregationType,
    Season,
)
from arpav_ppcv.schemas import coverages
from arpav_ppcv.schemas.coverages import CoverageInternal
from arpav_ppcv.schemas import observations

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

settings = get_settings()
session = sqlmodel.Session(db.get_engine(settings))
http_client = httpx.Client()

In [2]:
cov_id = "tas_30yr_anomaly_seasonal_agree_model_ensemble-30yr-model_ensemble-tas-anomaly-rcp26-tw1-DJF"
cov = CoverageInternal(
    identifier=cov_id,
    configuration=db.get_coverage_configuration_by_coverage_identifier(session, cov_id)
)
cov

CoverageInternal(configuration=CoverageConfiguration(netcdf_main_dataset_name='tas', observation_variable_id=UUID('d0a75169-1f50-43ff-aeb3-92bedabda68f'), thredds_url_pattern='ensembletwbc/std/clipped/tas_avgagree_anom_{time_window}_{scenario}_{year_period}_VFVGTAA.nc', observation_variable_aggregation_type=<ObservationAggregationType.SEASONAL: 'SEASONAL'>, name='tas_30yr_anomaly_seasonal_agree_model_ensemble', wms_main_layer_name='tas-uncertainty_group', uncertainty_lower_bounds_coverage_configuration_id=None, id=UUID('85b2feb9-a5fe-44b3-a447-1ccf712d3aa1'), wms_secondary_layer_name='tas', uncertainty_upper_bounds_coverage_configuration_id=None, display_name_english='Mean temperature', unit='ºC', related_time_series_coverage_configuration_id=UUID('b89a42c8-587f-43a3-9b4f-c39dbe9f5679'), display_name_italian='Temperatura media', palette='uncert-stippled/seq-YlOrRd', description_english='Mean temperature', color_scale_min=0.0, description_italian='Temperatura media', color_scale_max=6.0

In [3]:
ts_related = operations.get_time_series_related_coverage(cov)

In [4]:
ts_related.identifier

'tas_seasonal_absolute_model_ensemble-annual-model_ensemble-tas-absolute-rcp26-DJF'